In [1]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats

In [2]:
CLINICAL_TABLE_NUM = 3
melanoma_clinical_data = get_constant_cohort(CLINICAL_TABLE_NUM)
res_melanoma_clinical_data = melanoma_clinical_data.set_index('Patient id')
print(f'Num of samples in table is {len(melanoma_clinical_data)}')
display(melanoma_clinical_data.head(20))
res_melanoma_clinical_data['Melanoma type'].value_counts()

Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.xlsx

and labels:
/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx
Num of samples in table is 18


,index,Patient id,Melanoma type,response,prior to biopsy,after biopsy,ICI,BRAF
2,2,M99,Mucosal,NR,ICI,other therapy,True,False
9,9,M106,Cutaneous,NR,ICI,other therapy,True,False
10,10,M107,Cutaneous,NR,ICI,other therapy,True,False
14,14,M111,Mucosal,NR,ICI,other therapy,True,False
20,20,M120,Cutaneous,NR,ICI,other therapy,True,True
28,28,M128,Mucosal,NR,other therapy,ICI,True,False
30,30,M130,Cutaneous,R,other therapy,ICI,True,False
31,31,M131,Cutaneous,R,ICI,other therapy,True,False
36,36,M136,Mucosal,NR,ICI,other therapy,True,False
37,37,M137,Cutaneous,R,ICI,other therapy,True,True


Cutaneous    14
Mucosal       4
Name: Melanoma type, dtype: int64

In [3]:
sum(res_melanoma_clinical_data.BRAF)

7

In [5]:
n_samples=71
ICI=None
melanoma_type=None
prior_biopsy=None
after_biopsy=None,
only_metastasis_sample=False
response=None
therapy_translator=therapy_ICI_translator
CLINICAL_LABELS_PATH = r'/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx'
MELANOMA_CLINICAL_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.xlsx'
print(f'Using clinical table in path:\n {MELANOMA_CLINICAL_DATA_PATH}\n\nand labels:\n{CLINICAL_LABELS_PATH}')

#  Loads xlsx files
CLINICAL_LABELS_PATH = r'/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx'
MELANOMA_CLINICAL_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.xlsx'
print(f'Using clinical table in path:\n {MELANOMA_CLINICAL_DATA_PATH}\n\nand labels:\n{CLINICAL_LABELS_PATH}')

melanoma_clinical_data = pd.read_excel(MELANOMA_CLINICAL_DATA_PATH)
clinical_labels = pd.read_excel(CLINICAL_LABELS_PATH)
# takes only first 71 samples, fill Nan and creat dictionary mapping
melanoma_clinical_data = melanoma_clinical_data.iloc[:n_samples][
    ['Patient id', 'Clinical response', 'Melanoma type', 'Therapy(ies) prior to biopsy', 'Therapy after biopsy',
     'Primary=1; Metastasis=0', 'Genotype ']]
# fill Nans
melanoma_clinical_data['Melanoma type'] = melanoma_clinical_data['Melanoma type'].fillna('??')
melanoma_clinical_data['Clinical response'] = melanoma_clinical_data['Clinical response'].fillna('??')
# Take only samples that are metastasis (if requested)
if only_metastasis_sample:
    melanoma_clinical_data = melanoma_clinical_data[melanoma_clinical_data['Primary=1; Metastasis=0'] != 1]
melanoma_clinical_data = melanoma_clinical_data.drop(columns='Primary=1; Metastasis=0')
# Convert response label mapping using clinical_label table
labels_mapping = {v[0]: v[1] for v in clinical_labels[['Clinical response', 'binary label']].values}
labels_mapping['R '] = 'R'
labels_mapping['PD (NR; for pembro) NR for d+t'] = 'NR'
melanoma_clinical_data['response'] = melanoma_clinical_data['Clinical response'].apply(lambda x: labels_mapping[x])
# Convert melanoma type
melanoma_type_translate = {'Cutaneous': 'Cutaneous', 'Mucosal ': 'Mucosal', 'Uveal': 'other', 'UN primary': 'other',
                           'Acral': 'other', 'Unknown': 'other', '??': 'other'}
melanoma_clinical_data['Melanoma type'] = melanoma_clinical_data['Melanoma type'].apply(
    lambda rr: melanoma_type_translate[rr])
# Convert therapy prior/after biopsy
melanoma_clinical_data['prior to biopsy'] = melanoma_clinical_data['Therapy(ies) prior to biopsy'].apply(
    lambda x: therapy_translator(x))
melanoma_clinical_data['after biopsy'] = melanoma_clinical_data['Therapy after biopsy'].apply(
    lambda x: therapy_translator(x))
melanoma_clinical_data['ICI'] = (melanoma_clinical_data['prior to biopsy'] == 'ICI') | (
        melanoma_clinical_data['after biopsy'] == 'ICI')
melanoma_clinical_data = melanoma_clinical_data.drop(
    columns=['Therapy(ies) prior to biopsy', 'Therapy after biopsy', 'Clinical response'])


Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.xlsx

and labels:
/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx
Using clinical table in path:
 /storage/md_keren/shitay/Data/tables/Melanoma_clinical_data_12.21_unportected.xlsx

and labels:
/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx


In [12]:
# melanoma_clinical_data['Genotype '].apply(lambda x: True if 'BRAF' in x else False)
melanoma_clinical_data['Genotype '].astype(str).apply(lambda x: True if 'BRAF' in x else False)

0     False
1     False
2     False
3     False
4     False
      ...  
66    False
67    False
68    False
69    False
70    False
Name: Genotype , Length: 71, dtype: bool